# Dreadnode Video Logging

This notebook demonstrates how to log video data using Dreadnode's `Video` data type. The examples cover various video sources including file paths, numpy arrays for frame sequences, and MoviePy video clips.

## Features

- Log videos from file paths (MP4, MOV, etc.)
- Create and log videos from sequences of numpy array frames
- Log MoviePy video clip objects
- Control video properties like frame rate, dimensions and format

In [ ]:
import dreadnode as dn

dn.configure(
    server="Your Dreadnode API", # Replace with your server address
    token="Your Dreadnode API Key", # Replace with your token
    project="video-examples"
)

/Users/raja/Desktop/dreadnode/projects/sdk/.venv/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## 1. File Path Examples

The simplest way to log video is directly from file paths. Dreadnode supports common video formats like MP4, MOV, and others.

In [2]:
import numpy as np
from PIL import Image, ImageDraw
import tempfile
import imageio

def create_sample_mov(duration=3, fps=30, width=320, height=240):
    """Creates a simple 3-second .mov test video with a moving rectangle."""
    
    frames = []
    total_frames = int(duration * fps)
    
    for i in range(total_frames):
        img = Image.new('RGB', (width, height), color=(0, 0, 0))
        draw = ImageDraw.Draw(img)
        
        x_pos = int((i / total_frames) * (width - 60))
        y_pos = int((i / total_frames) * (height - 60))
        r = int(255 * (1 - i / total_frames))
        b = int(255 * (i / total_frames))
        
        draw.rectangle([x_pos, y_pos, x_pos + 60, y_pos + 60], fill=(r, 100, b))
        draw.text((10, 10), f"Frame {i}", fill=(255, 255, 255))
        
        frames.append(np.array(img))
    
    temp_file = tempfile.NamedTemporaryFile(suffix='.mov', delete=False)
    temp_file.close()
    
    imageio.mimwrite(temp_file.name, frames, fps=fps, codec='libx264', 
                    quality=7, pixelformat='yuv420p')
    
    return temp_file.name


video_file_path = create_sample_mov()
print(f"Sample MOV created at: {video_file_path}")

Sample MOV created at: /var/folders/ln/f1c_19n537310md15785b6d80000gn/T/tmpa5yfwmh2.mov


In [3]:
import dreadnode as dn
from dreadnode import Video

with dn.run("video_file_examples") as r:
    dn.log_input("video_file", Video(
        video_file_path,
        caption="Basic video file example"
    ))
    
    dn.log_input("format_override", Video(
        video_file_path,
        format="mp4",
        caption="MOV file with MP4 format override"
    ))
    
    dn.log_input("dimension_override", Video(
        video_file_path,
        width=1280,
        height=720,
        caption="Video with dimension specifications"
    ))

21:41:38.956 video_file_examples


/Users/raja/Desktop/dreadnode/projects/sdk/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Numpy Array Frames

You can create videos from numpy arrays representing individual frames. This is useful for visualizing generated content, simulations, or processing results.

In [4]:
import numpy as np

with dn.run("video_numpy_examples") as r:
    frame = np.ones((120, 160, 3), dtype=np.uint8) * 200  # Gray background
    
    frame[40:80, 60:100] = [255, 0, 0]  # Red rectangle
    
    static_frames = [frame.copy() for _ in range(3)]
    
    dn.log_input("static_video", Video(
        static_frames,
        fps=5,  # 5 frames per second
        format="mp4",
        caption="Static video with 3 identical frames"
    ))

21:41:39.210 video_numpy_examples
MoviePy - Building video /var/folders/ln/f1c_19n537310md15785b6d80000gn/T/tmp26q9pf3i.mp4.
MoviePy - Writing video /var/folders/ln/f1c_19n537310md15785b6d80000gn/T/tmp26q9pf3i.mp4



MoviePy - Done !
MoviePy - video ready /var/folders/ln/f1c_19n537310md15785b6d80000gn/T/tmp26q9pf3i.mp4


## 3. MoviePy Video Clips

MoviePy is a popular library for video editing in Python. Dreadnode supports logging VideoClip objects directly.

In [5]:
from moviepy.video.io.VideoFileClip import VideoFileClip

with dn.run("video_moviepy_examples") as r:
    video_clip = VideoFileClip(video_file_path)
    
    dn.log_input("moviepy_full", Video(
        video_clip,
        caption="Full video loaded with MoviePy"
    ))
    
    try:
        if hasattr(video_clip, 'subclip'):
            first_5_seconds = video_clip.subclip(0, 5)
            dn.log_input("moviepy_clip", Video(
                first_5_seconds,
                caption="First 5 seconds of video"
            ))
    except Exception as e:
        print(f"Subclip not supported in this MoviePy version: {e}")
    
    # Clean up resources
    video_clip.close()

21:41:39.235 video_moviepy_examples
{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'qt', 'minor_version': '512', 'compatible_brands': 'qt', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [320, 240], 'bitrate': 46, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': 'FFMP', 'encoder': 'Lavc61.19.100 libx264'}}], 'input_number': 0}], 'duration': 3.0, 'bitrate': 51, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [320, 240], 'video_bitrate': 46, 'video_fps': 30.0, 'video_duration': 3.0, 'video_n_frames': 90}
/Users/raja/Desktop/dreadnode/projects/sdk/.venv/lib/python3.12/site-packages/imageio_ffmpeg/binaries/ffmpeg-macos-aarch64-v7.1 -i /var/folders/ln/f1c_19n537310md15785b6d80

MoviePy - Done !
MoviePy - video ready /var/folders/ln/f1c_19n537310md15785b6d80000gn/T/tmpx47gkl1c.mp4


## Clean it up

In [6]:
import os
os.unlink(video_file_path)

## Summary

This notebook demonstrated how to log videos in Dreadnode from various sources:

1. Direct file paths (MP4, MOV, etc.)
2. Numpy arrays representing frame sequences
3. MoviePy VideoClip objects
